# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Открытие-файла-с-данными-и-изучение-общей-информации" data-toc-modified-id="Открытие-файла-с-данными-и-изучение-общей-информации-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Открытие файла с данными и изучение общей информации</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Предобработка данных</a></span><ul class="toc-item"><li><span><a href="#Обработка-пропусков" data-toc-modified-id="Обработка-пропусков-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Обработка пропусков</a></span></li><li><span><a href="#Замена-типа-данных" data-toc-modified-id="Замена-типа-данных-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Замена типа данных</a></span></li><li><span><a href="#Обработка-дубликатов" data-toc-modified-id="Обработка-дубликатов-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Обработка дубликатов</a></span></li><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Лемматизация</a></span></li><li><span><a href="#Категоризация-данных" data-toc-modified-id="Категоризация-данных-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Категоризация данных</a></span></li></ul></li><li><span><a href="#Ответим-на-актуальные-вопросы" data-toc-modified-id="Ответим-на-актуальные-вопросы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Ответим на актуальные вопросы</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

## Открытие файла с данными и изучение общей информации

In [1]:
import pandas as pd # импорт библиотеки Pandas
import statistics as st # импорт библиотеки статистики

data = pd.read_csv('/datasets/data.csv') # чтение файла с данными и сохранение его в переменную data

display(data.head(10)) # отражение первых 10 строк таблицы для обзорного анализа данных по столбцам, типам и характеру информации
display(data.tail(10)) # отражение последних 10 строк таблицы для обзорного анализа данных по столбцам, типам и характеру информации

data.info() # вывод сводной информации о массиве данных для анализа
data.columns # проверка перечня названий столбцов

display(data.sort_values(by='days_employed', ascending=True)) # проверка на наличие возможного нереалистичного минимального возможного стажа
display(data.sort_values(by='days_employed', ascending=False)) # проверка на наличие возможного нереалистичного максимального возможного стажа

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21515,1,-467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,-914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,-404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
16335,1,-18388.949901,61,среднее,1,женат / замужем,0,F,сотрудник,0,186178.934089,операции с недвижимостью
4299,0,-17615.563266,61,среднее,1,женат / замужем,0,F,компаньон,0,122560.741753,покупка жилья
7329,0,-16593.472817,60,высшее,0,женат / замужем,0,F,сотрудник,0,124697.846781,заняться высшим образованием
17838,0,-16264.699501,59,среднее,1,женат / замужем,0,F,сотрудник,0,51238.967133,на покупку автомобиля
16825,0,-16119.687737,64,среднее,1,женат / замужем,0,F,сотрудник,0,91527.685995,покупка жилой недвижимости
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6954,0,401755.400475,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,176278.441171,ремонт жилью
10006,0,401715.811749,69,высшее,0,Не женат / не замужем,4,F,пенсионер,0,57390.256908,получение образования
7664,1,401675.093434,61,среднее,1,женат / замужем,0,F,пенсионер,0,126214.519212,операции с жильем
2156,0,401674.466633,60,среднее,1,женат / замужем,0,M,пенсионер,0,325395.724541,автомобили
7794,0,401663.850046,61,среднее,1,гражданский брак,1,F,пенсионер,0,48286.441362,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


**Вывод**


Согласно документации к данным:
* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

В выведенной таблице на основе предоставленного массива данных обнаружены следующие особенности при предварительном экспресс-анализе:
 1. наименования столбцов отражены в соответствии с правилами представления имен в массивах python; 
 2. общее количество строк с данными 21525, в столбцах 'days_employed' и 'total_income' их по 19351, т.е. в данных присутствуют пропуски;
 3. недочеты в форматах отображения и типах данных: 
    - days_employed: занятость указана в днях в нецелочисленном формате с отрицательными значениями, кроме того в некоторых позициях указываются значения с сотнями тысяч рабочих дней стажа(что соответствует сотням лет стажа, что является некорректным), это может быть связано с технологическими ошибками при выгрузке данных, либо некорректным отражением стажа в базах данных и это необходимо дополнительно проанализировать и исключить нереалистичные экстремумы. Также тип данных необходимо конвертировать в целочисленный для удобства восприятия информации и отразить значения по модулю;
    - education_id: тип данных строки, рекомендуется отобразить прописным шрифтом.
 4. присутствие экстремальных минимумов и максимумов в отображающем стаж столбце 'days_employed', рекомендуется получение дополнительной информации от разработчиков для устранения возможных ошибок во входных данных.

## Предобработка данных

### Обработка пропусков

In [2]:
# подсчет пропусков
display(data.isna().sum()) # NaN
display(data.isnull().sum()) # '0'

# в данном блоке заполняются пустые ячейки с зарплатой медианными значениями по группам клиентов
# блок переменных для income
employee_income = []
employee_median_income = 0
companion_income = []
companion_median_income = 0
retiree_income = []
retiree_median_income = 0
gov_emp_income = []
gov_emp_median_income = 0
businessman_income = []
businessman_median_income = 0
unemployed_income = []
unemployed_median_income = 0
student_income = []
student_median_income = 0
decree_income = []
decree_median_income = 0

for i in range(len(data)):
    if (data['total_income'][i] > 0 and data['income_type'][i] == 'сотрудник'):
        employee_income.append(data['total_income'][i])
    if (data['total_income'][i] > 0 and data['income_type'][i] == 'компаньон'):
        companion_income.append(data['total_income'][i])
    if (data['total_income'][i] > 0 and data['income_type'][i] == 'пенсионер'):
        retiree_income.append(data['total_income'][i])
    if (data['total_income'][i] > 0 and data['income_type'][i] == 'госслужащий'):
        gov_emp_income.append(data['total_income'][i])
    if (data['total_income'][i] > 0 and data['income_type'][i] == 'предприниматель'):
        businessman_income.append(data['total_income'][i])
    if (data['total_income'][i] > 0 and data['income_type'][i] == 'безработный'):
        unemployed_income.append(data['total_income'][i])
    if (data['total_income'][i] > 0 and data['income_type'][i] == 'студент'):
        student_income.append(data['total_income'][i])
    if (data['total_income'][i] > 0 and data['income_type'][i] == 'в декрете'):
        decree_income.append(data['total_income'][i])
employee_median_income = st.median(employee_income)
companion_median_income = st.median(companion_income)
retiree_median_income = st.median(retiree_income)
gov_emp_median_income = st.median(gov_emp_income)
businessman_median_income = st.median(businessman_income)
unemployed_median_income = st.median(unemployed_income)
student_median_income = st.median(student_income)
decree_median_income = st.median(decree_income)
display('employee_median', employee_median_income)
display('companion_median', companion_median_income)
display('retiree_median', retiree_median_income)
display('gov_emp_median', gov_emp_median_income)
display('businessman_median',businessman_median_income)
display('unemployed_median', unemployed_median_income)
display('student_median', student_median_income)
display('decree_median', decree_median_income)

for j in range(len(data)):
    if (data['income_type'][j] == 'сотрудник' and (pd.isna(data['total_income'][j]) == True or pd.isnull(data['total_income'][j]) == True)):
        data['total_income'][j] = employee_median_income
    if (data['income_type'][j] == 'компаньон' and (pd.isna(data['total_income'][j]) == True or pd.isnull(data['total_income'][j]) == True)):
        data['total_income'][j] = companion_median_income
    if (data['income_type'][j] == 'пенсионер' and (pd.isna(data['total_income'][j]) == True or pd.isnull(data['total_income'][j]) == True)):
        data['total_income'][j] = retiree_median_income
    if (data['income_type'][j] == 'госслужащий' and (pd.isna(data['total_income'][j]) == True or pd.isnull(data['total_income'][j]) == True)):
        data['total_income'][j] = gov_emp_median_income
    if (data['income_type'][j] == 'предприниматель' and (pd.isna(data['total_income'][j]) == True or pd.isnull(data['total_income'][j]) == True)):
        data['total_income'][j] = businessman_median_income
    if (data['income_type'][j] == 'безработный' and (pd.isna(data['total_income'][j]) == True or pd.isnull(data['total_income'][j]) == True)):
        data['total_income'][j] = unemployed_median_income
    if (data['income_type'][j] == 'студент' and (pd.isna(data['total_income'][j]) == True or pd.isnull(data['total_income'][j]) == True)):
        data['total_income'][j] = student_median_income
    if (data['income_type'][j] == 'в декрете' and (pd.isna(data['total_income'][j]) == True or pd.isnull(data['total_income'][j]) == True)):
        data['total_income'][j] = decree_median_income

# в данном блоке заполняются пустые ячейки с количеством рабочих дней медианными значениями по группам клиентов        
# блок переменных для days_employed
employee_days_employed = []
employee_median_days_employed = 0
companion_days_employed = []
companion_median_days_employed = 0
retiree_days_employed = []
retiree_median_days_employed = 0
gov_emp_days_employed = []
gov_emp_median_days_employed = 0
businessman_days_employed = []
businessman_median_days_employed = 0
unemployed_days_employed = []
unemployed_median_days_employed = 0
student_days_employed = []
student_median_days_employed = 0
decree_days_employed = []
decree_median_days_employed = 0

for k in range(len(data)):
    if ((data['days_employed'][k] > 0 or data['days_employed'][k] < 0) and data['income_type'][k] == 'сотрудник'):
        employee_days_employed.append(data['days_employed'][k])
    if ((data['days_employed'][k] > 0 or data['days_employed'][k] < 0) and data['income_type'][k] == 'компаньон'):
        companion_days_employed.append(data['days_employed'][k])
    if ((data['days_employed'][k] > 0 or data['days_employed'][k] < 0) and data['income_type'][k] == 'пенсионер'):
        retiree_days_employed.append(data['days_employed'][k])
    if ((data['days_employed'][k] > 0 or data['days_employed'][k] < 0) and data['income_type'][k] == 'госслужащий'):
        gov_emp_days_employed.append(data['days_employed'][k])
    if ((data['days_employed'][k] > 0 or data['days_employed'][k] < 0) and data['income_type'][k] == 'предприниматель'):
        businessman_days_employed.append(data['days_employed'][k])
    if ((data['days_employed'][k] > 0 or data['days_employed'][k] < 0) and data['income_type'][k] == 'безработный'):
        unemployed_days_employed.append(data['days_employed'][k])
    if ((data['days_employed'][k] > 0 or data['days_employed'][k] < 0) and data['income_type'][k] == 'студент'):
        student_days_employed.append(data['days_employed'][k])
    if ((data['days_employed'][k] > 0 or data['days_employed'][k] < 0) and data['income_type'][k] == 'в декрете'):
        decree_days_employed.append(data['days_employed'][k])
employee_median_days_employed = st.median(employee_days_employed)
companion_median_days_employed = st.median(companion_days_employed) # ошибка
retiree_median_days_employed = st.median(retiree_days_employed)
gov_emp_median_days_employed = st.median(gov_emp_days_employed)
businessman_median_days_employed = st.median(businessman_days_employed)
unemployed_median_days_employed = st.median(unemployed_days_employed)
student_median_days_employed = st.median(student_days_employed)
decree_median_days_employed = st.median(decree_days_employed)
display('employee_median_days_employed', employee_median_days_employed)
display('companion_median_days_employed', companion_median_days_employed)
display('retiree_median_days_employed', retiree_median_days_employed)
display('gov_emp_median_days_employed', gov_emp_median_days_employed)
display('businessman_median_days_employed',businessman_median_days_employed)
display('unemployed_median_days_employed', unemployed_median_days_employed)
display('student_median_days_employed', student_median_days_employed)
display('decree_median_days_employed', decree_median_days_employed)

for l in range(len(data)):
    if (data['income_type'][l] == 'сотрудник' and (pd.isna(data['days_employed'][l]) == True or pd.isnull(data['days_employed'][l]) == True)):
        data['days_employed'][l] = employee_median_days_employed
    if (data['income_type'][l] == 'компаньон' and (pd.isna(data['days_employed'][l]) == True or pd.isnull(data['days_employed'][l]) == True)):
        data['days_employed'][l] = companion_median_days_employed
    if (data['income_type'][l] == 'пенсионер' and (pd.isna(data['days_employed'][l]) == True or pd.isnull(data['days_employed'][l]) == True)):
        data['days_employed'][l] = retiree_median_days_employed
    if (data['income_type'][l] == 'госслужащий' and (pd.isna(data['days_employed'][l]) == True or pd.isnull(data['days_employed'][l]) == True)):
        data['days_employed'][l] = gov_emp_median_days_employed
    if (data['income_type'][l] == 'предприниматель' and (pd.isna(data['days_employed'][l]) == True or pd.isnull(data['days_employed'][l]) == True)):
        data['days_employed'][l] = businessman_median_days_employed
    if (data['income_type'][l] == 'безработный' and (pd.isna(data['days_employed'][l]) == True or pd.isnull(data['days_employed'][l]) == True)):
        data['days_employed'][l] = unemployed_median_days_employed
    if (data['income_type'][l] == 'студент' and (pd.isna(data['days_employed'][l]) == True or pd.isnull(data['days_employed'][l]) == True)):
        data['days_employed'][l] = student_median_days_employed
    if (data['income_type'][l] == 'в декрете' and (pd.isna(data['days_employed'][l]) == True or pd.isnull(data['days_employed'][l]) == True)):
        data['days_employed'][l] = decree_median_days_employed


# подсчет/проверка наличия пропусков после замены
display(data.isna().sum()) # NaN
display(data.isnull().sum()) # '0'


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

'employee_median'

142594.39684740017

'companion_median'

...

'decree_median_days_employed'

-3296.7599620220594

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:138: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:134: SettingWithCopyWarning: 
A value is trying to 

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

# **Вывод**

Входные данные были проверены на наличие пропусков, обнаружено 2174 пропуска в столбцах 'days_employed' и 'total_income'.
Не все пропущенные значения влияют на исследование. Возможной причиной отсутствия данных  в указанных выше позициях в связанных друг с другом столбцах может быть заполнение анкет людьми, которые не работали/официально не были трудоустроены и не получали официальный доход соответственно. Достаточно заменить их медианой по категориям занятости каждой из целевых групп для исследования, чтобы не потерять массивы информации. 

### Замена типа данных

In [3]:
# воспользуемся методом astype для перевода вещественного формата данных в целочисленный и избавимся от минусов
try:
    data['days_employed'] = abs(data['days_employed'].astype('int64'))
    data['total_income'] = data['total_income'].astype('int64')
except:
    print('В данных по опыту присутствует другая необработанная ошибка, необходимо проанализировать столбец поподробнее')
    
display(data) # проверка на предмет наличия минусов
data.info() # проверка на предмет изменения типа данных в столбце

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21521,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Была произведена замена вещественного типа данных на целочисленный в стобце 'days_employed' и 'total_income' методом astype() с использованием конструкции try/except, т.к. стандартный метод to_numeric() мог бы быть использован исключительно в случаях с работой со строками, а в данном случае исходный тип данных вещественный, кроме того конструкция try/except помогает в случае появления возможных непредвиденных исходных данных(нечисловых типов) для используемого метода. По итогу изменения проведена визуальная проверка массива и проверка типа преобразованного столбца данных. Отрицательные значения были заменены на положительные методом abs() для удобства восприятия данных по опыту.

### Обработка дубликатов

In [4]:
# найдем дубликаты в столбцах education, family_status, gender, income_type, purpose методом value_counts()
display(data['education'].value_counts()) # начальная проверка по столбцу 'education'
data['education'] = data['education'].str.lower() # исправление
display(data['education'].value_counts()) # конечная проверка по столбцу 'education'

display(data['family_status'].value_counts()) # начальная проверка по столбцу 'family_status'
data['family_status'] = data['family_status'].str.lower() # исправление
display(data['family_status'].value_counts()) # конечная проверка по столбцу 'family_status'

display(data['gender'].value_counts()) # начальная проверка по столбцу 'gender'

display(data['income_type'].value_counts()) # начальная проверка по столбцу 'income_type'

display(data['purpose'].value_counts()) # начальная проверка по столбцу 'purpose'

display(data['total_income'].value_counts()) # начальная проверка по столбцу 'total_income'

# работа с явными дубликатами
duplicated_data = data[data.duplicated()].head(50)
display(duplicated_data)
data = data.drop_duplicates().reset_index(drop=True) # удаление дубликатов, перезагрузка индекса
duplicated_data_1 = data[data.duplicated()].head(50)
display(duplicated_data_1)

# воспользуемся методом astype для перевода вещественного формата данных в целочисленный и избавимся от минусов
try:
    data['days_employed'] = abs(data['days_employed'].astype('int64'))
    data['total_income'] = data['total_income'].astype('int64')
except:
    print('В данных по опыту присутствует другая необработанная ошибка, необходимо проанализировать столбец поподробнее')

data.info()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

...

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

142594    1105
172357     509
118514     414
150447     147
126262       3
          ... 
101387       1
138249       1
280240       1
390148       1
264193       1
Name: total_income, Length: 18608, dtype: int64

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,1574,41,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для семьи
3290,0,365213,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
4182,1,1574,34,высшее,0,гражданский брак,1,F,сотрудник,0,142594,свадьба
4851,0,365213,60,среднее,1,гражданский брак,1,F,пенсионер,0,118514,свадьба
5557,0,365213,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
6312,0,1574,30,среднее,1,женат / замужем,0,M,сотрудник,0,142594,строительство жилой недвижимости
7808,0,365213,57,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы
7921,0,365213,64,высшее,0,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы
7938,0,365213,71,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы
8583,0,365213,58,высшее,0,не женат / не замужем,4,F,пенсионер,0,118514,дополнительное образование


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

По итогам анализа на предмет наличия возможных недостатков в массиве данных были проверены следующие гипотезы и получен следующий результат по ним:
 1. в массиве данных 'education' обнаружены и исправлены дублирования связанные с регистрами, возможные причины: технологические ошибки, низкий уровень дисциплины в части заполнения анкет служащими, отсутсвие четких критериев по заполнению информации через онлайн/оффлайн формы потенциальными клиентами;
 2. в массиве данных 'family_status' обнаружено неудобное представление данных связанное с регистрами, исправлено понижением регистра;
 3. в массиве данных 'gender' не обнаружены дублирования, однако присутствуют некоторые уникальные значения в виде гендернонейтральных индивидов;
 4. в массиве данных 'income_type' обнаружены возможные дублирующие/пересекающиеся категории, такие как "компаньон" - "предприниматель", "сотрудник" - "госслужащий" - "в декрете". Данные категории возможно объединить, если получить дополнительную аналитическую связующию их информацию для коррекции групп данных;
 5. в массиве данных 'purpose' обнаружены возможные неявные дублирования, которые будут обработаны на последующем этапе работы над проектом по анализу данных;
 6. в массиве данных 'total_type' отражены ранее обнаруженные данные с пустыми нулевыми полями, возможно, при получении дополнительной аналитики данных людей можно поместить в "безработный", "пенсионер", "студент", "в декрете". 
 7. проверено наличие явных дубликатов, удаление явных дубликатов(71 строка) и перезагрузка индекса.

### Лемматизация

In [5]:
from pymystem3 import Mystem # импортируем pymystem3 для проведения лемматизации
from collections import Counter # импорт библиотеки counter для подсчета встречаемости значений в списках

m = Mystem() # получение лемматизатора для слов на русском

lemmatized_list = [] # пустой массив/словарь для записи лемм в него
lemmatized_list_all_cat = [] # массив для записи лемм с частотой их упоминания в столбце 'purpose'
lemmatized_list_all_cat_modif = [] # модификация словаря/массива в табличную структуру для читаемого вывода на экран для анализа

# цикл для перебора строк-целей получения кредитов по каждому запросу, выделение лемм
for row in data['purpose']:
    lemmatized_purpose = m.lemmatize(row)
    lemmatized_list.extend(lemmatized_purpose)
lemmatized_list_all_cat = Counter(lemmatized_list).most_common() # подсчет встречаемости значений в списке
lemmatized_list_all_cat_modif = pd.DataFrame(data=lemmatized_list_all_cat, columns=['lemma', 'rating']) # создание массива данных с леммами и их рейтингом для более удобного отображения в рамках анализа
display(lemmatized_list_all_cat_modif) # проверка отображения аналитической таблицы

,lemma,rating
0,,33570
1,\n,21454
2,недвижимость,6351
3,покупка,5897
4,жилье,4460
5,автомобиль,4306
6,образование,4013
7,с,2918
8,операция,2604
9,свадьба,2324


**Вывод**

На этапе лемматизации выделены леммы в значениях столбца с целями получения кредита, лемматизация проходила в несколько этапов:
* импортированы библиотеки pymystem3 и collections;
* получен лемматизатор и созданы пустые массивы для работы и размещения данных;
* циклом перебора столбца 'purpose' выделены леммы и сохранены в список;
* список лемм отсортирован от наиболее употребительных к наименее употребительным;
* создан массив данных с леммами и частотой их упоминания в столбце ''purpose'.

По ранжированному массиву данных можно сделать вывод, что наиболее часто встречающиеся цели для получения займов следующие(в порядке убывания приоритета):
 1. кредит на приобретение жилой недвижимости, ключевые слова: недвижимость, жилье, жилой;
 2. кредит на приобретение автомобиля, ключевые слова: автомобиль;
 3. кредит на образование, ключевые слова: образование;
 4. кредит на строительство или ремонт, ключевые слова: строительство, ремонт;
 5. кредит на свадьбу, ключевые слова: свадьба;
 6. кредит на приобретение коммерческой недвижимости, ключевые слова: недвижимость, коммерческий.

### Категоризация данных

In [6]:
# напишем функцию по категоризации по сформулированным выше категориям/целевым леммам
def group_aimes(row):
    purpose_group = row['purpose']
    lemmatize_group = m.lemmatize(purpose_group)
    if 'автомобиль' in lemmatize_group:
        return 'автомобиль'
    if 'образование' in lemmatize_group:
        return 'образование'
    if 'свадьба' in lemmatize_group:
        return 'свадьба'
    if ('строительство' in lemmatize_group) or ('ремонт' in lemmatize_group):
        return 'строительство и ремонт'
    if ('коммерческий' in lemmatize_group) & ('недвижимость' in lemmatize_group):
        return  'коммерческая недвижимость'
    if ('жилье' in lemmatize_group) or ('жилой' in lemmatize_group) or ('недвижимость' in lemmatize_group):
        return 'жилая недвижимость'
    return 'прочее'
        
data['group_aimes'] = data.apply(group_aimes, axis=1)
display(data['group_aimes'].value_counts())

# напишем функцию по категоризации по людям с возвратом кредитов в срок и с несоблюдением сроков возвратов
def group_debt(row):
    debt_group = row['debt']
    if debt_group == 1:
        return 'возврат кредита в срок'
    return 'возврат кредита не в срок'
        
data['group_debt'] = data.apply(group_debt, axis=1)
display(data['group_debt'].value_counts())

# напишем функцию по категоризации по наличию и отсутствию детей по каждому из заявителей
def group_children(row):
    children_group = row['children']
    if children_group > 0:
        return 'есть дети'
    return 'нет детей'
        
data['group_children'] = data.apply(group_children, axis=1)
display(data['group_children'].value_counts())

# напишем функцию по категоризации по зарплате по каждому из заявителей
# выделим 4 группы зарплат по уровням медиан
median_income = data['total_income'].median() # расчет базовой медианы по доходам
lower_median_income = data[data['total_income'] < median_income]['total_income'].median() # расчет медианы по доходам ниже базовой медианы, выделение 2-х низжиш подгрупп зарплат низкой, ниже среднего
upper_median_income = data[data['total_income'] > median_income]['total_income'].median() # расчет медианы по доходам выше базовой медианы, выделение 2-х высших подгрупп зарплат высокой, выше среднего
display('Медианная зарплата', median_income)
display('Медианная зарплата между низкой и зарплатой ниже медианной', lower_median_income)
display('Медианная зарплата между зарплатой выше мединной и высокой зарплатой', upper_median_income)

def group_income(row):
    income_group = row['total_income']
    if income_group < lower_median_income:
        return 'низкая зарплата'
    if (income_group >= lower_median_income) & (income_group < median_income):
        return 'зарплата ниже среднего'
    if (income_group >= median_income) & (income_group < upper_median_income):
        return 'зарплата выше среднего'
    if income_group >= upper_median_income:
        return 'высокая зарплата'
    return 'прочее'
        
data['group_total_income'] = data.apply(group_income, axis=1)
display(data['group_total_income'].value_counts())

жилая недвижимость           7015
автомобиль                   4306
образование                  4013
строительство и ремонт       2485
свадьба                      2324
коммерческая недвижимость    1311
Name: group_aimes, dtype: int64

возврат кредита не в срок    19713
возврат кредита в срок        1741
Name: group_debt, dtype: int64

нет детей    14138
есть дети     7316
Name: group_children, dtype: int64

'Медианная зарплата'

142594.0

'Медианная зарплата между низкой и зарплатой ниже медианной'

103460.0

'Медианная зарплата между зарплатой выше мединной и высокой зарплатой'

196593.0

зарплата выше среднего    6375
высокая зарплата          5306
зарплата ниже среднего    4887
низкая зарплата           4886
Name: group_total_income, dtype: int64

**Вывод**

Категоризация данных - это создание структур данных/словарей для часто используемых классификаций данных для упрощения и ускорения доступа к ним. Сформировав категории на основе словаря наиболее часто встречающихся целей для получения кредита были добавлены новые аналитики в столбце 'group_aimes'. Эти категории имеют наибольший охват в рамках поставленной задачи. Однако, есть предложения по первоначальному массиву данных - предоставить более расширенные данные из анкет/заявок для более детального анализа категории "недвижимость" с целью выявления подкатегорий.
В целом для формулировки последующих выводов и ответов на вопросы были сформированы следующие функции:
* по категориям/целевым леммам;
* по людям с возвратом кредитов в срок и с несоблюдением сроков возвратов;
* по наличию и отсутствию детей по каждому из заявителей;
* по зарплате по каждому из заявителей.

## Ответим на актуальные вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
# определим уровень просрочек в группе с детьми и в группе без детей
pivot_child_debt = data.pivot_table(index=['group_children'], columns='group_debt', values='debt', aggfunc='count')
pivot_child_debt['итого'] = pivot_child_debt['возврат кредита в срок'] + pivot_child_debt['возврат кредита не в срок']
pivot_child_debt['доля кредитов с просрочкой, в %'] = (pivot_child_debt['возврат кредита не в срок'] / pivot_child_debt['итого']) * 100
pivot_child_debt['доля кредитов без просрочки, в %'] = (pivot_child_debt['возврат кредита в срок'] / pivot_child_debt['итого']) * 100
display(pivot_child_debt)

group_debt,возврат кредита в срок,возврат кредита не в срок,итого,"доля кредитов с просрочкой, в %","доля кредитов без просрочки, в %"
group_children,,,,,
есть дети,677,6639,7316,90.746309,9.253691
нет детей,1064,13074,14138,92.474183,7.525817


**Вывод**

Сформулированы гипотезы:
1. Есть дети - кредитная дисциплина выше;
2. Нет детей - кредитная дисциплина ниже.

Для проверки гипотез сформирована сводная таблица по наличию детей и просрочкам. Гипотеза №1 имеет долю кредитов с просрочкой 90,7%, гипотеза №2 имеет долю кредитов с просрочкой 92,5%, что доказывает, что кредитная дисциплина выше при наличии детей в семье, хотя и незначительный перевес имеет данный фактор.



- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:
# определим уровень просрочек в группах по семейному положению
pivot_family_status_debt = data.pivot_table(index=['family_status'], columns='group_debt', values='debt', aggfunc='count')
pivot_family_status_debt['итого'] = pivot_family_status_debt['возврат кредита в срок'] + pivot_family_status_debt['возврат кредита не в срок']
pivot_family_status_debt['доля кредитов с просрочкой, в %'] = (pivot_family_status_debt['возврат кредита не в срок'] / pivot_family_status_debt['итого']) * 100
pivot_family_status_debt['доля кредитов без просрочки, в %'] = (pivot_family_status_debt['возврат кредита в срок'] / pivot_family_status_debt['итого']) * 100
display(pivot_family_status_debt)

group_debt,возврат кредита в срок,возврат кредита не в срок,итого,"доля кредитов с просрочкой, в %","доля кредитов без просрочки, в %"
family_status,,,,,
в разводе,85,1110,1195,92.887029,7.112971
вдовец / вдова,63,896,959,93.430657,6.569343
гражданский брак,388,3763,4151,90.652855,9.347145
женат / замужем,931,11408,12339,92.454818,7.545182
не женат / не замужем,274,2536,2810,90.249110,9.750890


**Вывод**

Сформирована сводная таблица по семейным статусам и просрочкам. Процент просрочек по категориям в разводе и вдовам/вдовцам 93,4%, по людям в гражданском браке 90,7%, по женатым/замужним 92,4%, по неженатым/незамужним 90,2% с просрочками. Таким образом семейный статус не показывает значительных расхождений в допущении просрочек по категориям. Однако, следует выделить холостых людей, у которых платежная дисциплина несколько выше, чем по остальным категориям.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [9]:
# определим уровень просрочек в группах по уровню дохода
pivot_group_total_income_debt = data.pivot_table(index=['group_total_income'], columns='group_debt', values='debt', aggfunc='count')
pivot_group_total_income_debt['итого'] = pivot_group_total_income_debt['возврат кредита в срок'] + pivot_group_total_income_debt['возврат кредита не в срок']
pivot_group_total_income_debt['доля кредитов с просрочкой, в %'] = (pivot_group_total_income_debt['возврат кредита не в срок'] / pivot_group_total_income_debt['итого']) * 100
pivot_group_total_income_debt['доля кредитов без просрочки, в %'] = (pivot_group_total_income_debt['возврат кредита в срок'] / pivot_group_total_income_debt['итого']) * 100
display(pivot_group_total_income_debt)

group_debt,возврат кредита в срок,возврат кредита не в срок,итого,"доля кредитов с просрочкой, в %","доля кредитов без просрочки, в %"
group_total_income,,,,,
высокая зарплата,377,4929,5306,92.894836,7.105164
зарплата выше среднего,552,5823,6375,91.341176,8.658824
зарплата ниже среднего,428,4459,4887,91.242071,8.757929
низкая зарплата,384,4502,4886,92.140810,7.859190


**Вывод**

Сформирована сводная таблица по уровням зарплат и просрочкам.
Определены следующие уровни зарплат путем разбиения массива зарплат по медианам: высокая зарплата(выше 196593 усл. ед.), зарплата выше среднего(выше 142594 усл. ед.), зарплата ниже среднего(ниже 142594 усл. ед.), низкая зарплата(ниже 103460 усл. ед.).
Процент просрочек по категориям с высокой зарплатой 92,8%, с зарплатой выше среднего 91,3%, с зарплатой ниже среднего 91,2%, с низкой зарплатой 92,1%. Таким образом разница в зарплатах не показывает значительных расхождений просрочек по категориям, однако можно выделить более высокую платежную дисциплину по категории с зарплатами выше средней медианной по выборке, но ниже средней медианной по выборке высоких зарплат.

- Как разные цели кредита влияют на его возврат в срок?

In [10]:
# определим уровень просрочек в группах по целям получения кредита
pivot_group_aimes_debt = data.pivot_table(index=['group_aimes'], columns='group_debt', values='debt', aggfunc='count')
pivot_group_aimes_debt['итого'] = pivot_group_aimes_debt['возврат кредита в срок'] + pivot_group_aimes_debt['возврат кредита не в срок']
pivot_group_aimes_debt['доля кредитов с просрочкой, в %'] = (pivot_group_aimes_debt['возврат кредита не в срок'] / pivot_group_aimes_debt['итого']) * 100
pivot_group_aimes_debt['доля кредитов без просрочки, в %'] = (pivot_group_aimes_debt['возврат кредита в срок'] / pivot_group_aimes_debt['итого']) * 100
display(pivot_group_aimes_debt)

group_debt,возврат кредита в срок,возврат кредита не в срок,итого,"доля кредитов с просрочкой, в %","доля кредитов без просрочки, в %"
group_aimes,,,,,
автомобиль,403,3903,4306,90.640966,9.359034
жилая недвижимость,504,6511,7015,92.815396,7.184604
коммерческая недвижимость,99,1212,1311,92.448513,7.551487
образование,370,3643,4013,90.779965,9.220035
свадьба,186,2138,2324,91.996558,8.003442
строительство и ремонт,179,2306,2485,92.796781,7.203219


**Вывод**

Сформирована сводная таблица по целям и просрочкам. 
% допущения просрочек от наибольшего к наименьшему:
* строительство и ремонт - 92,8%;
* жилая недвижимость - 92,8%;
* коммерческая недвижимость - 92,4%;
* свадьба - 91,9%;
* образование - 90,7%;
* автомобиль - 90,6%.

Наибольшая степень допущения просрочек в категориях "строительство и ремонт" и "жилая недвижимость", наименьшая в категориях "образование" и "автомобиль".

## Общий вывод

По итогам данного проекта были пройдены следующие основные этапы:
1. Изучение информации о входном массиве данных, определение его особенностей, закономерностей и недочетов;
2. Проведение предобработки данных, основные моменты: выявление устранимых и неустранимых недочетов в данных. Устранение всех возможных недочетов/артефактов, указание на необходимость предоставления дополнительной документации/информации по неустранимым недочетам, работа с пропусками, типами данных, дубликатами;
3. Лемматизация, составление рейтинга по частоте упортебления ключевых слов в потребностях людей при получении кредитов, формулировка полного перечня целей получения кредитов(кредит на приобретение жилой недвижимости, ключевые слова: недвижимость, жилье, жилой; кредит на приобретение автомобиля, ключевые слова: автомобиль; кредит на образование, ключевые слова: образование; кредит на строительство или ремонт, ключевые слова: строительство, ремонт; кредит на свадьбу, ключевые слова: свадьба; кредит на приобретение коммерческой недвижимости, ключевые слова: недвижимость, коммерческий);
4. Категоризация данных, а именно выделение словарей/структур данных для удобства обращения и упрощения представлений данных для заказчика. Категории: по категориям/целевым леммам; по людям с возвратом кредитов в срок и с несоблюдением сроков возвратов; по наличию и отсутствию детей по каждому из заявителей; по зарплате по каждому из заявителей;
5. Сформированы сводные аналитические таблицы и сделаны основные выводы и даны ответы на вопросы:
    * кредитная дисциплина выше при наличии детей в семье, хотя и незначительный перевес имеет данный фактор;
    * семейный статус не показывает значительных расхождений в допущении просрочек по категориям. Однако, следует выделить холостых людей, у которых платежная дисциплина несколько выше, чем по остальным категориям;
    * разница в зарплатах не показывает значительных расхождений просрочек по категориям, однако можно выделить более высокую платежную дисциплину по категории с зарплатами выше средней медианной по выборке, но ниже средней медианной по выборке высоких зарплат;
    * наибольшая степень допущения просрочек в категориях "строительство и ремонт" и "жилая недвижимость", наименьшая в категориях "образование" и "автомобиль".
    